In [ ]:
import DifferentialEquations as DE
println( Threads.nthreads() )

In [ ]:

trajN = 40

K = [-0.17197997940,];
0.5*(-1.0+sqrt(1+4*K[1]))

## Hénon map
$x^{\prime}=-y + 2(K-x^2)$

$y^{\prime}=x$

In [ ]:
function Henon_map!(x_new,x,K,t)
    x_new[1] = -x[2] + 2*(K[1]-x[1]^2)
    x_new[2] = x[1]
    nothing
end

x0s = Array{Float64,2}(undef, trajN, 2);
x0s[:,1] = range(-0.6, stop=0.0, length=trajN);
x0s[:,2] = range(-0.6, stop=0.0, length=trajN);

tspan = (0, 200)
# Henon_func= DE.DiscreteFunction(Henon_map!; jac=Henon_map_jac!)
Henon_prob = DE.DiscreteProblem(Henon_map!, x0s[1,:], tspan, K)

function prob_func(prob, i, repeat)
    DE.remake(prob, u0=x0s[i,:])
end
Henon_probs = DE.EnsembleProblem(Henon_prob, prob_func=prob_func)

In [ ]:
Henon_sols = DE.solve(
    Henon_probs, DE.EnsembleThreads(), trajectories=trajN,)

In [7]:
Js = [zeros(2,2) for i in (1:trajN)]
for i = 1:trajN
    Js[i][1,2] =-1.0
    Js[i][2,1] = 1.0
    Js[i][2,2] = 0.0
end

In [ ]:

function Henon_map_jac!(DT_new,DT, trajNo, t)
    Js[trajNo][1,1] = -2.0*Henon_sols[trajNo].u[t][1]
    # J[1,2] = -1.0
    # J[2,1] = 1.0
    # J[2,2] = 0.0
    
    DT_new[:,:] = Js[trajNo]*DT
    nothing
end

In [ ]:

DP0 = [1.0 0.0; 0.0 1.0]
# Henon_func= DE.DiscreteFunction(Henon_map!; jac=Henon_map_jac!)
Henon_jac_prob = DE.DiscreteProblem(Henon_map_jac!, DP0, tspan, K)

function prob_func(prob, i, repeat)
    DE.remake(prob, p=i,)
end
Henon_jac_probs = DE.EnsembleProblem(Henon_jac_prob, prob_func=prob_func)

In [ ]:
Henon_jac_sols = DE.solve(Henon_jac_probs, DE.EnsembleThreads(), trajectories=trajN, )

In [ ]:
Henon_jac_sols[39]